In [1]:
%pip install sentence-transformers datasets transformers[torch]

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer, InputExample, losses, util
from torch.utils.data import DataLoader
import random
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
dataset = pd.read_csv("data/processed/fine_tuning_dataset.csv")

In [4]:
#génération de paires négatives
topics = dataset["Topic_group"].unique()
negative_pairs = []

for row in dataset.iterrows():
    negative_pairs.append([row[1]["Document"], random.choice([t for t in topics if t != dataset.Topic_group[row[0]]])])


In [5]:
train_examples = [
    InputExample(texts = [row[1]["Document"], row[1]["Topic_group"]], label = 1.0)
    for row in dataset.iterrows()
]
for pair in negative_pairs:
    train_examples.append(
        InputExample(texts = pair, label = 0.0)
    )

In [6]:
model = SentenceTransformer('models/all-MiniLM-L6-v2')

In [7]:

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=32)
train_loss = losses.CosineSimilarityLoss(model)


In [8]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=10,
    warmup_steps=100,
    show_progress_bar=True
)


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

/home/alex/dev/DevfestDijon/.venv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
500,0.133500
1000,0.043600


In [9]:
model.save('models/fine_tuned_ticket_embedder_v5')